In [14]:
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.initializers import RandomNormal
import numpy as np

In [20]:
def getSequencesToSequences(inputMatrix, labels):
    offset = labels.shape[0]%timesteps
    if offset > 0: offset = timesteps - offset
    npad = ((offset+1, timesteps), (0, 0), (0, 0), (0, 0))
    inputMatrix = np.pad(inputMatrix[1:], pad_width=npad, mode='constant', constant_values=0)
    npad = ((offset+1, timesteps), (0, 0))
    inputLabels = np.pad(labels[:-1], pad_width=npad, mode='constant', constant_values=0)
    npad = ((timesteps+offset+1, 0), (0, 0))
    targets = np.pad(labels[1:], pad_width=npad, mode='constant', constant_values=0)
    outputLabels = np.zeros(targets.shape[:1]+(timesteps,)+targets.shape[1:])
    il = np.zeros_like(inputLabels)
    for i in range(0, inputLabels.shape[0], timesteps):
        il[i] = inputLabels[i]
    inputLabels = il
    for i in range(0, len(targets), timesteps):
        outputLabels[i] = targets[i:i+timesteps] 
    return inputMatrix, inputLabels, outputLabels

timesteps = 10
inputMatrix = np.random.rand(57,224,224,3)# np.array([[[[i, i, i]]] for i in range(57)])
labels = np.array([[i, i, i] for i in range(57)])

inputMatrix, inputLabels, outputLabels = getSequencesToSequences(inputMatrix, labels)
batch_size=1    
img_gen = TimeseriesGenerator(inputMatrix, outputLabels, length=timesteps, sampling_rate=1, stride=timesteps, batch_size=batch_size)
ang_gen = TimeseriesGenerator(inputLabels, outputLabels, length=timesteps, sampling_rate=1, stride=timesteps, batch_size=batch_size)
batch_01 = img_gen[0]
batch_0 = ang_gen[0]
inputFrames, y = batch_01
inputSeq, y = batch_0

In [35]:
y

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [1., 1., 1.],
        [2., 2., 2.],
        [3., 3., 3.],
        [4., 4., 4.],
        [5., 5., 5.],
        [6., 6., 6.]]])

In [15]:
import tensorflow as tf

In [17]:
l = RandomNormal(mean=y, stddev=0.05, seed=None)

In [18]:
l

In [21]:
n = 5

In [22]:
m = np.zeros((n,)+y.shape)
with tf.Session():
    for i in range(n):
        m[i] = l(y.shape).eval()

In [78]:
m.shape

(5, 1, 10, 3)

In [41]:
y.shape[:-1]

(1, 10)

In [57]:
yy = np.repeat(y[np.newaxis, ...], n, axis=0)

In [58]:
yy.shape

(5, 1, 10, 3)

In [61]:
yy[3] == y

array([[[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]])

In [42]:
r = np.zeros(y.shape[:-1]+(1,))

In [43]:
r

array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]])

In [62]:
np.average(np.abs(m - yy), axis=-1).shape

(5, 1, 10)

In [63]:
rt = -np.average(np.abs(m - yy), axis=-1) - np.max(np.abs(m - yy), axis=-1)

In [64]:
rt

array([[[-0.08360068, -0.04487692, -0.04801671, -0.08602771,
         -0.10097557, -0.05753732, -0.11846288, -0.12682223,
         -0.11370722, -0.08432992]],

       [[-0.09932037, -0.04812137, -0.09128053, -0.06689561,
         -0.09658355, -0.09452613, -0.10628947, -0.12478105,
         -0.09613132, -0.0839475 ]],

       [[-0.09153414, -0.06154102, -0.05533313, -0.06912199,
         -0.09124817, -0.1070087 , -0.08625436, -0.12028225,
         -0.09516319, -0.10839764]],

       [[-0.0929417 , -0.05844967, -0.05712595, -0.08325633,
         -0.12505345, -0.05964681, -0.12617008, -0.12541914,
         -0.09897995, -0.11122735]],

       [[-0.10906103, -0.06936556, -0.10556194, -0.094242  ,
         -0.11758381, -0.06167269, -0.09130041, -0.12903476,
         -0.08454641, -0.10865752]]])

In [83]:
rtt = np.mean(rt, axis=0)

In [84]:
rtt.shape

(1, 10)

In [85]:
rtt

array([[-0.09529158, -0.05647091, -0.07146365, -0.07990873, -0.10628891,
        -0.07607833, -0.10569544, -0.12526789, -0.09770562, -0.09931199]])

In [80]:
bt = rtt, axis=-1)

In [81]:
bt

array([-0.45674152])

In [82]:
(rt - bt).shape

(5, 1, 10)

In [1]:
from FC_RNN_Evaluater.Stateful_FC_RNN_Configuration import *

/home/mcicek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
vgg_model, full_model, modelID, preprocess_input = getFinalModel(timesteps = timesteps, lstm_nodes = lstm_nodes, lstm_dropout = lstm_dropout, lstm_recurrent_dropout = lstm_recurrent_dropout, 
                      num_outputs = num_outputs, lr = learning_rate, include_vgg_top = include_vgg_top, use_vgg16 = use_vgg16)

In [9]:
def get_gradients(model):
    """Return the gradient of every trainable weight in model

    Parameters
    -----------
    model : a keras model instance

    First, find all tensors which are trainable in the model. Surprisingly,
    `model.trainable_weights` will return tensors for which
    trainable=False has been set on their layer (last time I checked), hence the extra check.
    Next, get the gradients of the loss with respect to the weights.

    """
    weights = []
    for tensor in model.trainable_weights:
        print(tensor.name)
        if model.get_layer(tensor.name.split('/')[0]).trainable:
            weights.append(tensor)
    optimizer = model.optimizer

    return optimizer.get_gradients(model.total_loss, weights)

In [10]:
get_gradients(full_model)

lstm_1/kernel:0
lstm_1/recurrent_kernel:0
lstm_1/bias:0
time_distributed_1/kernel:0
time_distributed_1/bias:0


[<tf.Tensor 'gradients/AddN_5:0' shape=(4099, 4096) dtype=float32>,
 <tf.Tensor 'gradients/AddN_4:0' shape=(1024, 4096) dtype=float32>,
 <tf.Tensor 'gradients/AddN_3:0' shape=(4096,) dtype=float32>,
 <tf.Tensor 'gradients/time_distributed_1/while/MatMul/Enter_grad/b_acc_3:0' shape=(1024, 3) dtype=float32>,
 <tf.Tensor 'gradients/time_distributed_1/while/BiasAdd/Enter_grad/b_acc_3:0' shape=(3,) dtype=float32>]

In [26]:
w = full_model.get_weights()

In [28]:
len(w)

35

In [29]:
type(w)

list

In [30]:
w[-1]

array([0., 0., 0.], dtype=float32)

In [32]:
w[-2].shape

(1024, 3)

In [33]:
w[-2]

array([[-0.0243619 , -0.06632722,  0.0723459 ],
       [ 0.04879396, -0.03963432,  0.01553073],
       [ 0.04456819, -0.07137289, -0.06297336],
       ...,
       [-0.00244952, -0.00757853,  0.0511878 ],
       [-0.06938513, -0.0087788 ,  0.07334001],
       [-0.0620813 , -0.04785295, -0.05943701]], dtype=float32)

In [29]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as k
from keras import losses
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from math import sqrt

model = full_model
inputs = inputFrames
outputs = model.predict([inputFrames, inputSeq])
targets = y
print(outputs.shape, targets.shape)
rmse = sqrt(mean_squared_error(targets[0], outputs[0]))
loss = losses.mean_squared_error(targets, model.output)

#  ===== Symbolic Gradient =====
gradients = k.gradients(loss, model.trainable_weights)

print("===BEFORE WALKING DOWN GRADIENT===")
print("outputs:\n", outputs)
print("targets:\n", targets)

# Begin TensorFlow
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

steps = 100  # steps of gradient descent
for s in range(steps):
    #print(model.input)
    # ===== Numerical gradient =====
    evaluated_gradients = sess.run(gradients, feed_dict={'tdCNN_input:0': inputFrames, 'aux_input:0': inputSeq})

    # Step down the gradient for each layer
    for i in range(len(model.trainable_weights)):
        sess.run(tf.assign_sub(model.trainable_weights[i], evaluated_gradients[i]))

    # Every 10 steps print the RMSE
    if s % 10 == 0:
        outputs = model.predict([inputFrames, inputSeq])
        rmse = sqrt(mean_squared_error(targets[0], outputs[0]))
        print("step " + str(s) + " rmse:", rmse)

final_outputs = model.predict([inputFrames, inputSeq])
final_rmse = sqrt(mean_squared_error(targets[0], outputs[0]))

print("===AFTER STEPPING DOWN GRADIENT===")
print("outputs:\n", outputs)
print("targets:\n", targets)

(1, 10, 3) (1, 10, 3)
===BEFORE WALKING DOWN GRADIENT===
outputs:
 [[[-5.8046376e+15 -5.8782211e+15 -5.8528937e+15]
  [-7.0172963e+15 -7.1062532e+15 -7.0756333e+15]
  [-7.2892150e+15 -7.3816180e+15 -7.3498116e+15]
  [-7.3316278e+15 -7.4245693e+15 -7.3925777e+15]
  [-7.3374416e+15 -7.4304566e+15 -7.3984393e+15]
  [-7.3382292e+15 -7.4312523e+15 -7.3992328e+15]
  [-7.3383344e+15 -7.4313607e+15 -7.3993401e+15]
  [-7.3383494e+15 -7.4313768e+15 -7.3993562e+15]
  [-7.3383516e+15 -7.4313768e+15 -7.3993562e+15]
  [-7.3383516e+15 -7.4313768e+15 -7.3993568e+15]]]
targets:
 [[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [1. 1. 1.]
  [2. 2. 2.]
  [3. 3. 3.]
  [4. 4. 4.]
  [5. 5. 5.]
  [6. 6. 6.]]]
step 0 rmse: 18.499424244125578


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').